In [1]:
%load_ext autoreload
%autoreload 2

In [28]:

import os as o

from pathlib import Path
import time
import crosspy as xpy
import numpy as np
import pyfftw
import numexpr as ne

t0=time.time()

folder_path=Path(r'/Users/tom/Documents/GitHub/crosspy/data/Tom')
Images = xpy.Imset(folder_path,'tif',[0,1])

# # fft filter settings: high pass, high pass width, low pass, low pass width
filter_settings=[4,2,15,8]
roi_1stpass = dict(size_pass = 300, overlap_percentage = 75, xcf_mesh=400)

# # build the dic class (but don't run it yet):
d=xpy.DIC(Images[0,1],roi_1stpass,filter_settings)


In [29]:
def get_subset(ims,size_pass,ss_locations,n,imno):
    #grab a subset from a stacked numpy array of images already loaded in RAM (imno is number in stack, n is subset number in ss_locations)
    
    subset=ims[ss_locations[n,0]:(ss_locations[n,0]+int(size_pass)),ss_locations[n,1]:(ss_locations[n,1]+int(size_pass)),imno]

    # if norm==True:
    #     #normalise
    subset_norm=(subset-np.ones_like(subset)*np.mean(subset))/np.std(subset)
    # else:
    #     subset_norm=np.array(subset) 

    return subset_norm

In [30]:
def plan_ffts(d,ffttype='fftw_numpy'):
    
    subset_shape=d.roi[0]
    #xcf_mesh=

    a = pyfftw.empty_aligned((subset_shape, subset_shape), dtype='complex128')
    b = pyfftw.empty_aligned((subset_shape, subset_shape), dtype='complex128')
    c = pyfftw.empty_aligned((2*subset_shape, 2*subset_shape), dtype='complex128')
    d = pyfftw.empty_aligned((2*subset_shape, 2*subset_shape), dtype='complex128')

    # Over both axes
    forward_fft = pyfftw.FFTW(a, b, axes=(0,1))
    inverse_fft = pyfftw.FFTW(a,b ,axes=(0,1),direction='FFTW_BACKWARD')
    forward_xcf_fft1 = pyfftw.FFTW(c,d, axes=(0,1))
    inverse_xcf_fft1 = pyfftw.FFTW(c,d, axes=(0,1),direction='FFTW_BACKWARD')

    prepared_ffts=[forward_fft,inverse_fft,forward_xcf_fft1,inverse_xcf_fft1]

    if ffttype=='fftw_numpy':
        prepared_ffts=[pyfftw.interfaces.numpy_fft.fft2,pyfftw.interfaces.numpy_fft.ifft2,pyfftw.interfaces.numpy_fft.fft2,pyfftw.interfaces.numpy_fft.ifft2]
    elif ffttype=='fftw_scipy':
        prepared_ffts=[pyfftw.interfaces.scipy_fftpack.fft2,pyfftw.interfaces.scipy_fftpack.ifft2,pyfftw.interfaces.scipy_fftpack.fft2,pyfftw.interfaces.scipy_fftpack.ifft2]
    else:
        prepared_ffts=[np.fft.fft2,np.fft.ifft2,np.fft.fft2,np.fft.ifft2]

    return prepared_ffts

In [31]:
#grab the reference and test subsets, and get subpixel registration
subset1=get_subset(d.ims,d.roi[0],d.ss_locations,123,0)
subset2=get_subset(d.ims,d.roi[0],d.ss_locations,123,1)

prepared_ffts=plan_ffts(d)

In [32]:
def freg(ROI_test,ROI_ref,XCF_roisize,XCF_mesh,data_fill,prepared_ffts):
    
    #FREG Register two FFTs to subpixel accuracy
    #a reduced form of the code submitted to the matlab file exchange
    #http://www.mathworks.co.uk/matlabcentral/fileexchange/18401-efficient-subpixel-image-registration-by-cross-correlation
    
    #reported in the literature:
    #Manuel Guizar-Sicairos, Samuel T. Thurman, and James R. Fienup
    #"Efficient subpixel image registration algorithms," Opt. Lett. 33, 156-158 (2008).
    
    #modified to handle the filtered FFT sizes
    #TBB 2012
    
    #ported to python
    #TPM 2020

    forward_fft=prepared_ffts[0]
    inverse_fft=prepared_ffts[1]
    forward_xcf_fft1=prepared_ffts[2]
    inverse_xcf_fft1=prepared_ffts[3]
    
    CC=np.zeros((XCF_roisize*2,XCF_roisize*2),dtype='complex')
    red_roisize=len(data_fill)

    ind1=int(XCF_roisize-np.floor(red_roisize/2))
    ind2=int(XCF_roisize+np.floor((red_roisize-1)/2))+1
    
    #fft shifting required to make it cross-correlation rather than convolution
    f=np.fft.fftshift(ROI_test*np.conj(ROI_ref))
    CC[ind1:ind2,ind1:ind2]= f

    #inverse FFT
    CC=inverse_xcf_fft1(CC)
    
    #get the maximum of the XCF and its location
    chi=np.amax(CC)
    loc=np.argmax(CC)
    rloc,cloc=np.unravel_index(loc,CC.shape)

    #get the shift in the original pixel grid from position of XCF peak
    XCF_roisize2=2*XCF_roisize

    if rloc > XCF_roisize:
        row_shift = rloc - XCF_roisize2
    else:
        row_shift = rloc

    if cloc > XCF_roisize:
        col_shift = cloc - XCF_roisize2
    else:
        col_shift = cloc
    
    row_shift = row_shift/2
    col_shift = col_shift/2

    #DFT computation
    row_shift = round(row_shift*XCF_mesh)/XCF_mesh
    col_shift = round(col_shift*XCF_mesh)/XCF_mesh
    dftshift = np.floor(np.ceil(XCF_mesh*1.5)/2)

    #matrix multiply DFT around current shift estimate
    roff=dftshift-row_shift*XCF_mesh
    coff=dftshift-col_shift*XCF_mesh

    #computer kernels and obtain DFT by matrix products
    prefac=-1j*2*np.pi/(XCF_roisize*XCF_mesh)

    #speed up kernel generation for reduced filtered FFT
    c1=[i for i in range(0,(XCF_roisize))]
    c_i=np.fft.ifftshift(c1).T - np.floor(XCF_roisize/2)
    c_i=c_i[data_fill]
    c_i=np.expand_dims(c_i,1)

    r_i=np.fft.ifftshift(c1) - np.floor(XCF_roisize/2)
    r_i=r_i[data_fill]
    r_i=np.expand_dims(r_i,0)

    m=np.array([i for i in range(0,int(np.ceil(XCF_mesh*1.5)))])
    m1=np.expand_dims(m,0)
    m2=np.expand_dims(m,1)

    arg1=(c_i)@(m1-coff)
    arg2=(m2-roff)@(r_i)
    
    kernc=ne.evaluate('exp(prefac*arg1)')
    kernr=ne.evaluate('exp(prefac*arg2)')
    kern = ROI_ref*ne.evaluate('conj(ROI_test)')
    arg3=kernr@kern@kernc
    CC2 = ne.evaluate('conj(arg3)')

    #locate maximum and map back to original pixel grid
    CCmax=np.abs(np.amax(CC2))
    loc=np.argmax(CC2)
    loc1,loc2=np.unravel_index(loc,CC2.shape)

    rloc=loc1-dftshift-1
    cloc=loc2-dftshift-1

    row_shift=row_shift+rloc/XCF_mesh
    col_shift=col_shift+cloc/XCF_mesh

    #normalise output
    bf1=np.sum(ROI_test.flatten()*np.conjugate(ROI_test.flatten()))
    bf2=np.sum(ROI_ref.flatten()*np.conjugate(ROI_ref.flatten()))

    return col_shift, row_shift, CCmax/np.sqrt(float(bf1*bf2))

In [ ]:
forward_fft=prepared_ffts[0]
inverse_fft=prepared_ffts[1]

roi=d.roi    
fftfil=d.fftfilter
hfil=d.hfilter
filters_settings=d.filter_settings

#h-filter the subsets and generate the fft filter
subset1_filt=hfil*subset1
subset2_filt=hfil*subset2

#FFT the subsets
f_s1=forward_fft(subset1_filt)
f_s2=forward_fft(subset2_filt)

fill1=(filters_settings[2]+filters_settings[3])
fill2=(roi[0]-(filters_settings[2]+filters_settings[3]-1))
fill3=roi[0]
data_fill=np.array([i for i in range(fill1)]+[i for i in range(fill2-1,fill3)])

#grab the reduced filter
fftfil_red=fftfil[data_fill,:]
fftfil_red=fftfil_red[:,data_fill]

#grab the reduced subsets
f_s1_red=f_s1[data_fill,:]
f_s1_red=f_s1_red[:,data_fill]

f_s2_red=f_s2[data_fill,:]
f_s2_red=f_s2_red[:,data_fill]

#perform the filtering (point-wise multiplication)
ROI_ref=f_s1_red*fftfil_red
ROI_test=f_s2_red*fftfil_red

In [34]:
%timeit col_shift, row_shift, CCmax = freg(ROI_test,ROI_ref,roi[0],roi[2],data_fill,prepared_ffts)

/Users/tom/opt/anaconda3/envs/tpm/lib/python3.7/site-packages/ipykernel_launcher.py:106: ComplexWarning: Casting complex values to real discards the imaginary part


23.4 ms ± 685 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
XCF_roisize=roi[0]
XCF_roimesh=roi[2]

print(XCF_roisize)
print(XCF_roimesh)

300
400
